In [10]:
import numpy as np
import matplotlib.pyplot as plt

import os
import sys

import skimage as skimage

import seaborn as sns

import re

import cv2 as cv2

from glob import glob

### Remove all CSV files

In [11]:
os.system('rm *.csv')

rm: cannot remove '*.csv': No such file or directory


256

In [12]:
%config InlineBackend.figure_format ='retina'

plt.rcParams.update({'font.size': 25})

In [13]:
window_size_list = [4, 10, 20, 50]

In [14]:
# which_folder = 'Healthy'

which_folder = 'RightSide'

# which_folder = 'LeftSide'

In [15]:
string = './' + which_folder + '/*/'

folder_list = glob(string, recursive = True)

len(folder_list)

18

In [16]:
folder_list

['./Healthy/H84N3-2/',
 './Healthy/H70N2-1/',
 './Healthy/H70N1-1/',
 './Healthy/H70N2-2/',
 './Healthy/H84N3-3/',
 './Healthy/H84N4-4/',
 './Healthy/H70N3-1/',
 './Healthy/H84N4-2/',
 './Healthy/H84N4-3/',
 './Healthy/H84N2-3/',
 './Healthy/H84N2-1/',
 './Healthy/H70N1-3/',
 './Healthy/H84N4-1/',
 './Healthy/H84N3-4/',
 './Healthy/H70N1-2/',
 './Healthy/H84N3-1/',
 './Healthy/H70N2-3/',
 './Healthy/H84N4-5/']

In [17]:
for i in range(len(folder_list)):

    files = os.listdir(folder_list[i])

In [18]:
files

['Result_W_4,H84N4-5.txt',
 'Orientation_W_20,H84N4-5.npy',
 'Coherance_W_10,H84N4-5.npy',
 'EigenValues_W_4,H84N4-5.npy',
 'Coherance_W_4,H84N4-5.npy',
 'EigenVectors_W_4,H84N4-5.npy',
 'EigenVectors_W_50,H84N4-5.npy',
 'EigenValues_W_50,H84N4-5.npy',
 'Result_W_10,H84N4-5.pdf',
 'EigenVectors_W_20,H84N4-5.npy',
 'Result_W_50,H84N4-5.pdf',
 'Result_W_50,H84N4-5.txt',
 'Result_W_10,H84N4-5.txt',
 'Coherance_W_50,H84N4-5.npy',
 'Structure_Tensor_W_10,H84N4-5.npy',
 'Structure_Tensor_W_20,H84N4-5.npy',
 'EigenValues_W_10,H84N4-5.npy',
 'Result_W_20,H84N4-5.txt',
 'Coherance_W_20,H84N4-5.npy',
 'Result_W_4,H84N4-5.pdf',
 'EigenVectors_W_10,H84N4-5.npy',
 'Structure_Tensor_W_50,H84N4-5.npy',
 'H84N4-5.tif',
 'Structure_Tensor_W_4,H84N4-5.npy',
 'Orientation_W_10,H84N4-5.npy',
 'EigenValues_W_20,H84N4-5.npy',
 'Orientation_W_50,H84N4-5.npy',
 'Result_W_20,H84N4-5.pdf',
 'OrientationCoherance2D.py',
 'Orientation_W_4,H84N4-5.npy']

### Isolate files of a certain window only

In [30]:
for window in window_size_list:
    
    ############################################################
    
    array = []
    
    for i in range(len(folder_list)):

        ############################################################

        keyword = 'Orientation_W_' + str(window) + ','

        file_of_interest = []

        for fname in os.listdir(folder_list[i]):

            if keyword in fname:

                file_of_interest.append(fname)

        file_string = folder_list[i] + file_of_interest[0]

        Orientation_temp = np.load(file_string)
        
        Orientation = Orientation_temp.copy()

#         Orientation = cv2.normalize(Orientation_temp[~np.isnan(Orientation_temp)], None, 
#                                  alpha = 1e-4, beta = 180, 
#                                  norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)

        ############################################################

        keyword = 'Coherance_W_' + str(window) + ','

        file_of_interest = []

        for fname in os.listdir(folder_list[i]):

            if keyword in fname:

                file_of_interest.append(fname)

        file_string = folder_list[i] + file_of_interest[0]

        Coherance_temp = np.load(file_string)
        
        Coherance = Coherance_temp[~np.isnan(Coherance_temp)].copy()

#         Coherance = cv2.normalize(Coherance_temp[~np.isnan(Coherance_temp)], None, 
#                                  alpha = 1e-4, beta = 1, 
#                                  norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F)

        ############################################################

        histogram_coherance = plt.hist(Coherance, bins = 2, weights = np.ones(len(Coherance)) / len(Coherance));

        plt.close()

        low_coherance, high_coherance = 100 * histogram_coherance[0]

        ############################################################
        
        if (which_folder == 'Healthy'):

            array.append(np.asarray((folder_list[i][10:][:-1], 
                                     np.nanmean(Orientation.ravel()), 
                                     np.nanmedian(Orientation.ravel()), 
                                     np.nanstd(Orientation.ravel()),
                                     np.m(Coherance.ravel()), 
                                     np.nanmedian(Coherance.ravel()), 
                                     np.nanstd(Coherance.ravel()),
                                     low_coherance, high_coherance)))
            
        else:
            
            array.append(np.asarray((folder_list[i][12:][:-1], 
                                     np.nanmean(Orientation.ravel()), 
                                     np.nanmedian(Orientation.ravel()), 
                                     np.nanstd(Orientation.ravel()),
                                     np.nanmean(Coherance.ravel()), 
                                     np.nanmedian(Coherance.ravel()), 
                                     np.nanstd(Coherance.ravel()),
                                     low_coherance, high_coherance)))

        ############################################################

    np.savetxt(which_folder + '_Window_' + str(window) + '.csv', 
           array,
           fmt = "%s", delimiter = ',', 
           header = "Specimen, Mean Orientation, Median Orientation, StdDev Orientation, \
                       Mean Coherance, Median Coherance, StdDev Coherance, \
                        %Low Coherance, %High Coherance")